In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import exists
from os import remove
import json
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import sys
sys.path.insert(0, "../util/")
import util as util

%autosave 5

Autosaving every 5 seconds


# Adding dummies and train-test-split

In [2]:
datingTrainWithDummiesExists = exists('../data/datingTrainWithDummies.csv')
datingTestWithDummiesExists = exists('../data/datingTestWithDummies.csv')
datingTrainWithoutDummiesExists = exists('../data/datingTrainWithoutDummies.csv')
datingTestWithoutDummiesExists = exists('../data/datingTestWithoutDummies.csv')
datingFullWithDummiesExists = exists('../data/datingFullWithDummies.csv')
datingFullWithoutDummiesExists = exists('../data/datingFullWithoutDummies.csv')

relatedDummiesDictionaryExists = exists('../data/relatedDummiesDictionary.json')

In [3]:
with open('../data/columnDataDictionary.json') as d:
    columnDataDictionary = json.load(d)
columnList = columnDataDictionary['columnList']
nonBinaryCategoricalList = columnDataDictionary['nonBinaryCategoricalList']
stringToFloatList = columnDataDictionary['stringToFloatList']
pointDistributionList = columnDataDictionary['pointDistributionList']
partnerList = columnDataDictionary['partnerList']

In [4]:
if (datingTrainWithDummiesExists and datingTestWithDummiesExists and datingFullWithDummiesExists and
    datingTrainWithoutDummiesExists and datingTestWithoutDummiesExists and datingFullWithoutDummiesExists and
    relatedDummiesDictionaryExists):
    datingTrainWithDummies = pd.read_csv('../data/datingTrainWithDummies.csv')
    datingTestWithDummies = pd.read_csv('../data/datingTestWithDummies.csv')
    datingTrainWithoutDummies = pd.read_csv('../data/datingTrainWithoutDummies.csv')
    datingTestWithoutDummies = pd.read_csv('../data/datingTestWithoutDummies.csv')
    datingFullWithDummies = pd.read_csv('../data/datingFullWithDummies.csv')
    datingFullWithoutDummies = pd.read_csv('../data/datingFullWithoutDummies.csv')
    
    with open('../data/relatedDummiesDictionary.json') as d:
        relatedDummiesDictionary = json.load(d)
    for df in [datingTrainWithDummies,datingTestWithDummies,datingFullWithDummies,datingTrainWithoutDummies,datingTestWithoutDummies,datingFullWithoutDummies]:
        df['zipcode'] = df['zipcode'].apply(str)
        if 'zipcode_o' in list(df.columns):
            df['zipcode_o'] = df['zipcode_o'].apply(str)
        for col in nonBinaryCategoricalList:
            if col in list(df.columns):
                df[col] = df[col].apply(str)
        
else:
    if (datingTrainWithDummiesExists):
        remove('../data/datingTrainWithDummies.csv')
    if (datingTestWithDummiesExists):
        remove('../data/datingTestWithDummies.csv')
    if (datingTrainWithoutDummiesExists):
        remove('../data/datingTrainWithoutDummies.csv')
    if (datingTestWithoutDummiesExists):
        remove('../data/datingTestWithoutDummies.csv')
    if (datingFullWithDummiesExists):
        remove('../data/datingFullWithDummies.csv')
    if (datingFullWithoutDummiesExists):
        remove('../data/datingFullWithoutDummies.csv')
    if (relatedDummiesDictionaryExists):
        remove('../data/relatedDummiesDictionary.json')
    
    !jupyter nbconvert --execute eda.ipynb
    
    datingData = pd.read_csv('../data/encoded-SpeedDatingData.csv')
    blindDateData = datingData[columnList]
    
    for col in stringToFloatList:
        blindDateData[col] = blindDateData[col].str.replace(',', '').astype(float)
    blindDateData['zipcode'] = blindDateData['zipcode'].str.replace(',', '')
    for col in nonBinaryCategoricalList:
        blindDateData[col] = blindDateData[col].apply(str)
    
    blindDateCategoricalData = blindDateData.select_dtypes(include=['O'])
    for col in blindDateCategoricalData.columns:
        blindDateData[col]=blindDateData[col].fillna('nan')
    relatedDummiesDictionary = {}
    for col in blindDateCategoricalData.columns:
        dummyData = pd.get_dummies(blindDateData[col],prefix=col,drop_first=True)
        for dummyCol in dummyData.columns:
            relatedDummiesDictionary[str(dummyCol)] = list(dummyData.columns)
            if col in partnerList:
                partnerList.append(str(dummyCol))
        blindDateData = pd.concat([blindDateData,dummyData],axis=1)
    with open('../data/relatedDummiesDictionary.json', 'w') as fp:
        json.dump(relatedDummiesDictionary, fp)
        
    partnerList = list(set(partnerList))
    columnDataDictionary = {"columnList": columnList,
                        "nonBinaryCategoricalList": nonBinaryCategoricalList,
                        "stringToFloatList": stringToFloatList,
                        "pointDistributionList": pointDistributionList,
                        "partnerList": partnerList}

    with open('../data/columnDataDictionary.json', 'w') as fp:
            json.dump(columnDataDictionary, fp)
    
    datingFullWithDummies = blindDateData.copy()
    match = datingFullWithDummies['match']
    X = datingFullWithDummies.drop(['match'], axis=1)
    
    datingTrainWithDummies, datingTestWithDummies, matchTrain, matchTest = train_test_split(X, match, test_size=0.2)
    
    datingTrainWithDummies['match'] = matchTrain
    datingTestWithDummies['match'] = matchTest
    
    datingTrainWithDummies.to_csv('../data/datingTrainWithDummies.csv',index=False)
    datingTestWithDummies.to_csv('../data/datingTestWithDummies.csv',index=False)
    datingFullWithDummies.to_csv('../data/datingFullWithDummies.csv',index=False)
    
    dummyColumns = list(relatedDummiesDictionary.keys())
    datingTrainWithoutDummies = datingTrainWithDummies.drop(dummyColumns, axis=1)
    datingTestWithoutDummies = datingTestWithDummies.drop(dummyColumns, axis=1)
    datingFullWithoutDummies = datingFullWithDummies.drop(dummyColumns, axis=1)
    
    datingTrainWithoutDummies.to_csv('../data/datingTrainWithoutDummies.csv',index=False)
    datingTestWithoutDummies.to_csv('../data/datingTestWithoutDummies.csv',index=False)
    datingFullWithoutDummies.to_csv('../data/datingFullWithoutDummies.csv',index=False)

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Data Cleaning

### Join partner

In [5]:
for partnerCol in partnerList:
    if (("_o" not in str(partnerCol)) and (partnerCol+"_o" not in datingTrainWithoutDummies.columns) and (partnerCol+"_o" not in datingTrainWithDummies.columns)):
        partnerWithDummies = datingFullWithDummies.copy()
        partnerWithoutDummies = datingFullWithoutDummies.copy()
        datingTrainWithDummies = util.joinToPartner(datingTrainWithDummies,partnerWithDummies)
        datingTrainWithoutDummies = util.joinToPartner(datingTrainWithoutDummies,partnerWithoutDummies)
        datingTrainWithDummies.to_csv('../data/datingTrainWithDummies.csv',index=False)
        datingTrainWithoutDummies.to_csv('../data/datingTrainWithoutDummies.csv',index=False)
        break

### Get distance

In [6]:
if (("partnerDistance" not in datingTrainWithoutDummies.columns) or 
    ("partnerDistance" not in datingTrainWithDummies.columns)):
    datingTrainWithoutDummies = util.returnDFWithpartnerDistance(datingTrainWithoutDummies,"train",True)
    datingTrainWithDummies["partnerDistance"] = datingTrainWithoutDummies["partnerDistance"]
    datingTrainWithDummies.to_csv('../data/datingTrainWithDummies.csv',index=False)
    datingTrainWithoutDummies.to_csv('../data/datingTrainWithoutDummies.csv',index=False)

09:27:40
0 of 6702: 0.0% complete
0.0% of data is None
09:32:41
100 of 6702: 1.4920919128618322% complete
0.3133393017009848% of data is None
09:37:41
200 of 6702: 2.9841838257236644% complete
0.5819158460161146% of data is None
09:42:39
299 of 6702: 4.461354819456878% complete
0.925096985974336% of data is None
09:47:41
399 of 6702: 5.953446732318711% complete
1.1936735302894659% of data is None
09:52:40
498 of 6702: 7.430617726051925% complete
1.4920919128618322% of data is None
09:57:38
597 of 6702: 8.907788719785138% complete
1.805431214562817% of data is None
10:02:40
697 of 6702: 10.39988063264697% complete
2.0590868397493285% of data is None
10:07:39
796 of 6702: 11.877051626380185% complete
2.5663980901223513% of data is None
10:12:41
896 of 6702: 13.369143539242017% complete
2.909579230080573% of data is None
10:17:40
995 of 6702: 14.846314532975232% complete
3.327364965681886% of data is None
10:22:41
1095 of 6702: 16.338406445837062% complete
3.640704267382871% of data is No

RateLimiter caught an error, retrying (0/2 tries). Called with (*('france',), **{}).
Traceback (most recent call last):
  File "/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 382, in _make_request
    self._valida

14:27:41
5965 of 6702: 89.0032826022083% complete
18.516860638615338% of data is None
14:32:40
6064 of 6702: 90.48045359594151% complete
18.830199940316323% of data is None
14:37:39
6163 of 6702: 91.95762458967472% complete
19.173381080274545% of data is None
14:42:40
6263 of 6702: 93.44971650253656% complete
19.531483139361384% of data is None
14:47:39
6362 of 6702: 94.92688749626977% complete
19.859743360190986% of data is None
14:52:41
6462 of 6702: 96.41897940913161% complete
20.21784541927783% of data is None
14:57:40
6561 of 6702: 97.89615040286482% complete
20.516263801850194% of data is None
15:02:39
6660 of 6702: 99.37332139659803% complete
20.99373321396598% of data is None


### Fix ambiguous scores

In [7]:
halfwayChangeColumnsFromWithDummies = [str(col) if (("1_s" in str(col)) | ("3_s" in str(col))) for col in datingTrainWithDummies]
halfwayChangeColumnsFromWithoutDummies = [str(col) if (("1_s" in str(col)) | ("3_s" in str(col))) for col in datingTrainWithoutDummies]

if(len(halfwayChangeColumnsFromWithDummies) + len(halfwayChangeColumnsFromWithoutDummies) > 0):
    for question in pointDistributionList:
        questionCols = [str(col) if (question in datingTrainWithDummies)]
        datingTrainWithDummies[f'{question}_sum'] = datingTrainWithDummies[questionCols].sum(axis = 1)
        datingTrainWithoutDummies[f'{question}_sum'] = datingTrainWithoutDummies[questionCols].sum(axis = 1)
        for questionCol in questionCols:
            datingTrainWithDummies[str(questionCol)] = datingTrainWithDummies[str(questionCol)] * 100 / datingTrainWithDummies[f'{question}_sum']
            datingTrainWithoutDummies[str(questionCol)] = datingTrainWithoutDummies[str(questionCol)] * 100 / datingTrainWithoutDummies[f'{question}_sum']

SyntaxError: invalid syntax (<ipython-input-7-80fedd74ddb2>, line 1)

### Replace Nans

# Training

## Logistic Regression

## KNN

## Random Forest Classifier

## Boosting

# Individual Testing

## Logistic Regression

## KNN

## Random Forest Classifier

## Boosting

# Ensemble Testing